<a href="https://colab.research.google.com/github/ozakbas/Computer-Vision-Experiments/blob/main/Convolutional_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Train model

In [ ]:
!unzip "/content/drive/MyDrive/train_data.zip" -d "/content"

In [ ]:
n_batches = 32

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = '/content/train_data'

train_generator = ImageDataGenerator(rescale=1./255, 
                                     validation_split=0.2,
                                     rotation_range=5,
                                     width_shift_range=0.1,
                                     height_shift_range=0.1,
                                     shear_range=0.1,
                                     brightness_range=(0.5,1.5)       
                                     )


train_batches = train_generator.flow_from_directory(
                                              TRAIN_DIR, 
                                              class_mode="categorical", 
                                              batch_size=n_batches, 
                                              target_size=(100,100),
                                              color_mode='grayscale',
                                              shuffle=True,
                                              seed=10,
                                              subset='training'
                                              
                                              )
val_batches = train_generator.flow_from_directory(
                                              TRAIN_DIR, 
                                              class_mode="categorical", 
                                              batch_size=n_batches, 
                                              target_size=(100,100),
                                              color_mode='grayscale',
                                              shuffle=True,
                                              seed=10,
                                              subset='validation'
                                              )

Found 23200 images belonging to 29 classes.
Found 5800 images belonging to 29 classes.


In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout

model = Sequential([
  Conv2D(filters=64, kernel_size=3, strides=1, activation='relu', input_shape=(100,100,1)),
  MaxPooling2D(pool_size=(2, 2), strides=1, padding='same'),
  Conv2D(filters=128, kernel_size=3, strides=1, activation='relu'),
  MaxPooling2D(pool_size=(2, 2), strides=1, padding='same'),
  Conv2D(filters=128, kernel_size=3, strides=2, activation='relu'),
  MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
  Conv2D(filters=128, kernel_size=3, strides=2, activation='relu'),
  MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
  Conv2D(filters=32, kernel_size=3, strides=2, activation='relu'),
  MaxPooling2D(pool_size=(2, 2), strides=2, padding='same'),
  Flatten(),
  Dense(29, activation='softmax')
])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 98, 98, 64)        640       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 98, 98, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 96, 96, 128)       73856     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 96, 96, 128)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 47, 47, 128)       147584    
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 24, 24, 128)       0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 11, 11, 128)      

In [ ]:
import os
from keras.callbacks import ModelCheckpoint

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

steps_per_epoch = int(23200 / n_batches)
validation_steps = int(5800 / n_batches)
'''
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


# Create a callback that saves the model's weights
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
'''
history = model.fit(
    train_batches,
    steps_per_epoch=steps_per_epoch,
    epochs=50,
    validation_data=val_batches,
    validation_steps=validation_steps,
    #callbacks=[cp_callback]
    )

model.save('my_model.h5')

In [ ]:
from google.colab import files
files.download('my_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

# Load trained model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('my_model.h5')

# Test

In [ ]:
!unzip "/content/drive/MyDrive/nonlabeled-test2.zip" -d "/content"

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os

TEST_DIR = '/content/nonlabeled-test2'

test_dataset = []

for index, filename in enumerate(sorted(os.listdir(TEST_DIR), key=lambda filename: int(filename.split('.')[0]) )):
  image = load_img(TEST_DIR + "/" + filename, target_size=(100,100), color_mode='grayscale')
  input_arr = img_to_array(image)
  input_arr /= 255.
  test_dataset.append(input_arr)

test_dataset = np.array(test_dataset)


In [ ]:
import numpy as np
from numpy import load 
labels = load('/content/drive/MyDrive/CS412_DATA/labels.npy')
np.unique(labels)

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'del', 'nothing', 'space'], dtype='<U7')

In [ ]:
from sklearn.preprocessing import LabelBinarizer
import pandas as pd

encoder = LabelBinarizer()
encoded_labels = encoder.fit_transform(labels)
encoded_labels = encoded_labels.astype(float)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset,  encoded_labels, verbose=2)

454/454 - 10s - loss: 0.0336 - accuracy: 0.9884


In [ ]:
pred = np.argmax(model.predict(test_dataset), axis=-1)
label = np.argmax(encoded_labels,axis = 1)

print(pred) 
print(label)

[ 9 22  9 ... 19 25 20]
[ 9 22  9 ... 19 25 20]


In [ ]:
categories = ['A' ,'B' ,'C' ,'D' ,'E' ,'F' ,'G' ,'H' ,'I' ,'J' ,'K' ,'L' ,'M' ,'N' ,'O' ,'P' ,'Q' ,'R',
              'S' ,'T' , 'U' ,'V' ,'W' ,'X' ,'Y' ,'Z' , 'del', 'nothing', 'space']
categories = np.array(categories)

convert = lambda x: categories[x]
reversed_labels = convert(pred)

In [ ]:
import pandas as pd

df = pd.DataFrame()
df['Prediction'] = reversed_labels
df['Id'] = df.index
df = df[df.columns[::-1]]
df

,Id,Prediction
0,0,J
1,1,W
2,2,J
3,3,M
4,4,M
...,...,...
14495,14495,Q
14496,14496,F
14497,14497,T
14498,14498,Z


In [ ]:
df.to_csv('submission.csv', index=False)
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>